In [2]:
import pandas as pd # python에서 데이터를 조작할 때 쓰는 툴
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
cate_depth = 1
train_df = pd.read_pickle("soma_goods_train.df")
vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer()

In [3]:
# set d_list, cate_list

title_list1 = []
title_list2 = []
cate_list1 = []
cate_list2 = []
idx_list = []
for each in train_df.iterrows():
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    
    if each[1]['cate1'] == u'디지털/가전' :
        idx_list.append(1)
        title_list1.append(each[1]['name'])
        cate_list1.append(cate)
    else :
        idx_list.append(2)
        title_list2.append(each[1]['name'])
        cate_list2.append(cate)

In [4]:
# make cate_dict

cate_dict1 = dict(zip(list(set(cate_list1)),range(len(set(cate_list1))))) # 카테고리 별로 id를 만들어주는 코드.
cate_dict2 = dict(zip(list(set(cate_list2)),range(len(set(cate_list2)))))
joblib.dump(cate_dict1,'cate_dict1.dat')
joblib.dump(cate_dict2,'cate_dict2.dat')

print cate_dict1[u'디지털/가전;네트워크장비;KVM스위치']
print cate_dict2[u'패션의류;남성의류;정장']

0
2


In [5]:
# make y_list

y_list1 = []
y_list2 = []
idx1 = 0
idx2 = 0
for i in range(0,10000) :
    if idx_list[i] == 1 :
        y_list1.append(cate_dict1[cate_list1[idx1]])
        idx1 = idx1 + 1
    else :
        y_list2.append(cate_dict2[cate_list2[idx2]])
        idx2 = idx2 + 1

In [6]:
# define removeStopWords

mapping = { 'A/S':'AS', u'현대백화점':u' 백화점 ', u'롯데백화점':u' 백화점 ', u'대구백화점':u' 백화점 ', 'IntelIntel':' Intel ', 'IBMIBM':' IBM ',
            u'남성':u' 남성 남성 남성 ', u'여성':u' 여성 여성 여성 ', u'남아':u' 아동 아동 아동 ', u'여아':u' 아동 아동 아동 ', u'한복':u' 한복 한복 한복 '
          }

stop_words = [u"%즉시할인쿠폰",  u"5% 즉시할인", u"0% 즉시할인", u"% 즉시할인", u"5% 할인", u"0% 할인", u"5%할인", u"0%할인", u"% 할인", u"적립",
              u"쿠폰", u"즉시",u"정품", u"증정", u"할인", u"하나 더", u"하나더", u"사은품", u"중고",
              u"해외직배송", u"직배송", u"무료배송", u"안전배송", u"당일", u"배송", u"발송", u"빠른", u"저렴한", u"새로운",
              u"신한카드", u"BC카드", u"KB국민카드", u"국민카드", u"롯데카드", u"KB카드", u"현대카드", u"삼성카드", 
              u"비씨카드", u"우리카드", u"제휴카드", u"하나카드", u"씨티카드",
              u"일까지"]
symbols = ["[","]","(",")", "+", "-", "/", "?", "~", "*", u"☆", u"★", u"●", u"♥", u"■", u"▶", u"◀", u"↑"]
ambg_words = [u"해외", u"구매대행"]

start = 0
length = 1
remove_ambg = True

# define concatenate function
def concate(list1, list2) :
    for each in list2:
        list1.append(each)
    return list1

remove_list = concate(stop_words, symbols)
if remove_ambg:
    remove_list = concate(remove_list,ambg_words)
    
def removeStopWords(word) :
     # replace
    for before, after in mapping.iteritems():
        word = word.replace(before, after)
    #remove
    for each in remove_list :
        word = word.replace(each," ")
    return word

In [7]:
# define splitMorph using konlpy

from konlpy.tag import Twitter
from konlpy.utils import pprint
twit = Twitter()

def splitMorph(word) :
    morphs = twit.pos(word)
    
    words = ""
    for morph in morphs :
        word = morph[0]
        words += " " + word
    use_nouns = False
    if use_nouns :
        nouns = twit.nouns(word)
        for noun in nouns :
            words += " " + noun
    return words

In [8]:
# remove all stop words
# And split all morphs

length = len(title_list1)
for i in range(0, length):
    title_list1[i] = removeStopWords(title_list1[i])
    title_list1[i] = splitMorph(title_list1[i])

length = len(title_list2)
for i in range(0, length):
    title_list2[i] = removeStopWords(title_list2[i])
    title_list2[i] = splitMorph(title_list2[i])    
    

In [9]:
# JSON result 미리 읽어오기

list_caffe = []
list_caffe = joblib.load('list_caffe.dat')
length = len(list_caffe)
print length

10000


In [10]:
########### append string
size = 10000
use_caffe = True
if use_caffe :
    idx1 = 0
    idx2 = 0
    for num in range(0, size):
        res = list_caffe[num]['result']
        
        if res[0] :
            if idx_list[num] == 1:
                title = title_list1[idx1]
            else :
                title = title_list2[idx2]
                for each in res[1] :   
                    title += " " + each[0].replace(" ", "")
            for each in res[2] :
                title += " " + each[0].replace(" ", "")
                
            if idx_list[num] == 1:
                title_list1[idx1] = title
                idx1 = idx1 + 1
            else :
                title_list2[idx2] = title
                idx2 = idx2 + 1
                
                
print title_list1[10]
# print train_df[test_num-1:test_num]

 Intel Pentium Dual Core G 2030 LGA 1155 Processor BX 80637 G 2030 toiletry lotion container covering binder


In [11]:
######### training set과 test set 나누기
title_list1_test = []
title_list2_test = []
title_list1_train = []
title_list2_train = []
y_list1_test = []
y_list2_test = []
y_list1_train = []
y_list2_train = []
size = 10000
idx1 = 0
idx2 = 0
for i in range(0,size) :
    if i % 10 == 0:
        if idx_list[i] == 1:
            title_list1_test.append(title_list1[idx1])
            y_list1_test.append(y_list1[idx1])
            idx1 = idx1 + 1
        else :
            title_list2_test.append(title_list2[idx2])
            y_list2_test.append(y_list2[idx2])
            idx2 = idx2 + 1
    else :
        if idx_list[i] == 1:
            title_list1_train.append(title_list1[idx1])
            y_list1_train.append(y_list1[idx1])
            idx1 = idx1 + 1
        else :
            title_list2_train.append(title_list2[idx2])
            y_list2_train.append(y_list2[idx2])
            idx2 = idx2 + 1
print len(title_list1_test)
print len(y_list2_train)

600
3600


In [12]:
##############################################

save = True
if save :
    joblib.dump(title_list1_test, "title_list1_test.dat")
    joblib.dump(title_list1_train, "title_list1_train.dat")
    joblib.dump(title_list2_test, "title_list2_test.dat")
    joblib.dump(title_list2_train, "title_list2_train.dat")

    joblib.dump(y_list1_test, "y_list1_test.dat")
    joblib.dump(y_list1_train, "y_list1_train.dat")
    joblib.dump(y_list2_test, "y_list2_test.dat")
    joblib.dump(y_list2_train, "y_list2_train.dat")
        

In [1]:
#############################################
# load
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib

title_list1_test = joblib.load("title_list1_test.dat")
title_list1_train = joblib.load("title_list1_train.dat")
title_list2_test = joblib.load("title_list2_test.dat")
title_list2_train = joblib.load("title_list2_train.dat")

y_list1_test = joblib.load("y_list1_test.dat")
y_list1_train = joblib.load("y_list1_train.dat")
y_list2_test = joblib.load("y_list2_test.dat")
y_list2_train = joblib.load("y_list2_train.dat")

vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer()

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
xtidf_transformer1 = TfidfTransformer()
xtidf_transformer2 = TfidfTransformer()

# create x_list
x_list1 = vectorizer1.fit_transform(title_list1_train)
x_list2 = vectorizer2.fit_transform(title_list2_train)
# x_list1 = xtidf_transformer1.fit_transform(x_list1)
# x_list2 = xtidf_transformer2.fit_transform(x_list2)


print x_list1.shape
print x_list2.shape

(5400, 10262)
(3600, 8060)


In [3]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
import numpy as np
#svc_param = {'C':np.logspace(-2,0,20)} 
svc_param = {'C':np.logspace(-2,0,10)} 
gs_svc1 = GridSearchCV(LinearSVC(loss='l2'),svc_param,cv=5,n_jobs=4) 
gs_svc2 = GridSearchCV(LinearSVC(loss='l2'),svc_param,cv=5,n_jobs=4) 

In [4]:
gs_svc1.fit(x_list1, y_list1_train)
gs_svc2.fit(x_list2, y_list2_train)

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' w

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.01668,  0.02783,  0.04642,  0.07743,  0.12915,
        0.21544,  0.35938,  0.59948,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [5]:
print gs_svc1.best_params_, gs_svc1.best_score_
print gs_svc2.best_params_, gs_svc2.best_score_
# overfitting underfitting 때문에.
clf1 = LinearSVC(C=gs_svc1.best_params_['C'])
clf2 = LinearSVC(C=gs_svc2.best_params_['C'])
# C 값이 overfitting을 결정한다.
clf1.fit(x_list1,y_list1_train)  # fit 하면 학습이 됨.
clf2.fit(x_list2,y_list2_train)  # fit 하면 학습이 됨.

{'C': 0.077426368268112694} 0.718333333333
{'C': 0.046415888336127774} 0.675


LinearSVC(C=0.046415888336127774, class_weight=None, dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)

In [6]:
# test
size = len(title_list1_test)
sum = 0
for num in range (0, size):
    title = title_list1_test[num]
    vec = vectorizer1.transform([title])
    pred = clf1.predict(vec)[0]
    answer = y_list1_test[num]
    if pred == answer:
        sum = sum + 1
print float(sum)/float(size)

size = len(title_list2_test)
sum = 0
for num in range (0, size):
    title = title_list2_test[num]
    vec = vectorizer2.transform([title])
    pred = clf2.predict(vec)[0]
    answer = y_list2_test[num]
    if pred == answer:
        sum = sum + 1
print float(sum)/float(size)

0.815
0.79


In [7]:
# 만든 모델 저장
joblib.dump(clf1,'classify_test1.model',compress=3)
joblib.dump(vectorizer1,'vectorizer_test1.dat',compress=3)
joblib.dump(clf2,'classify_test2.model',compress=3)
joblib.dump(vectorizer2,'vectorizer_test2.dat',compress=3)

['vectorizer_test2.dat']